In [2]:
import pandas as pd
import numpy as np
import requests
import json
import time
from bs4 import BeautifulSoup

In [5]:
url = 'https://api.yelp.com/v3/businesses/search'
response = requests.get(url)

f = open("./apikey.txt", "r")
MY_API_KEY = (f.read())

In [6]:
def yelp_data(search_term, zipcode, limit, url, api_key):
    headers = {'Authorization': 'Bearer {}'.format(MY_API_KEY),}

    url_params = {
                'term': search_term.replace(' ', '+'),
                'location': 'New York, NY, {zipcode}',
                'limit': limit}
    a = requests.get(url, headers = headers, params= url_params)
    b = a.text
    df = json.loads(b)
    return pd.DataFrame(df['businesses'])

In [7]:
def yelp_dataframe(search_term, list_of_zipcodes, limit, url, api_key):
    main = []
    maindf = pd.DataFrame()
    for i in range(0, len(list_of_zipcodes)):
        main.append(yelp_data(search_term, i, limit, url, api_key))
    for i in main:
        maindf = pd.concat([i, maindf], axis = 0)
    return maindf

In [88]:
testdf = yelp_dataframe('Restaurants', [10001, 10002, 10003, 10007], 50, url, MY_API_KEY)

In [123]:
def get_features(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    l = soup.find_all(class_= "lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--semibold__373c0__2l0fe text-size--large__373c0__3t60B")
    v = []
    for i in l:
        v.append(i.text.strip())
    return v

In [128]:
w = [get_features('https://www.yelp.com/biz/robertas-brooklyn-2?osq=robertas'), get_features('https://www.yelp.com/biz/maite-brooklyn?osq=maite'), get_features('https://www.yelp.com/biz/samesa-brooklyn?osq=samesa')]

In [129]:
w

[['Outdoor seating',
  'Delivery',
  'Takeout',
  'Sit-down dining',
  'Curbside pickup',
  'Masks required',
  'Staff wears masks',
  'Staff wears gloves',
  'Limited capacity',
  'Health Score',
  'Offers Delivery',
  'Offers Takeout'],
 ['Outdoor seating',
  'Delivery',
  'Takeout',
  'Sit-down dining',
  'Staff wears masks',
  'Hand sanitizer provided',
  'Sanitizing between customers',
  'Social distancing enforced',
  'Health Score',
  'Takes Reservations',
  'Offers Takeout'],
 ['Health Score', 'Takes Reservations', 'Offers Delivery', 'Offers Takeout']]

In [126]:
def outdoor_seating(array_of_lists):
    r = []
    for i in array_of_lists:
        if 'Outdoor seating' in i:
            r.append(1)
        else:
            r.append(0)
    return r

In [130]:
outdoor_seating(w)

[1, 1, 0]